In [21]:
import torch
import os
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torch.optim as optim
import json
from PIL import Image
from torchvision.transforms import Resize, ToTensor, Normalize

In [6]:
path_train= "./annotations/instances_train2017.json"
file = open(path)
anns = json.load(file)
anns.keys()
anns['annotations'][0]

{'segmentation': [[239.97,
   260.24,
   222.04,
   270.49,
   199.84,
   253.41,
   213.5,
   227.79,
   259.62,
   200.46,
   274.13,
   202.17,
   277.55,
   210.71,
   249.37,
   253.41,
   237.41,
   264.51,
   242.54,
   261.95,
   228.87,
   271.34]],
 'area': 2765.1486500000005,
 'iscrowd': 0,
 'image_id': 558840,
 'bbox': [199.84, 200.46, 77.71, 70.88],
 'category_id': 58,
 'id': 156}

## Data Preprocessing

In [23]:




coco_root_train = os.path.join( "./coco_dataset_test/train2017")
coco_root_val = os.path.join( "./coco_dataset_test/val2017")
coco_root_test = os.path.join("./coco_dataset_test/test2017")
coco_root_unlabeled = os.path.join("./coco_dataset_test/unlabeled2017")


# transform to be applieds to the images

transform = transforms.Compose([
    ToTensor(),
    Resize((500,333))
    ])

# Function to create CocoDataset instances
def create_coco_dataset(root, ann_file, transform=None):
    return CocoDetection(root=root, annFile=ann_file, transform=transform)

coco_dataset_train = create_coco_dataset(root=coco_root_train, ann_file=("./annotations/instances_train2017.json"), transform=transform)
coco_dataset_val = create_coco_dataset(root=coco_root_val, ann_file=("./annotations/instances_val2017.json"), transform=transform)
captions_coco_dataset_train = create_coco_dataset(root=coco_root_train, ann_file=("./annotations/captions_train2017.json"), transform=transform)
captions_coco_dataset_val = create_coco_dataset(root=coco_root_val, ann_file=("./annotations/captions_val2017.json"), transform=transform)
person_keypoint_coco_dataset_train = create_coco_dataset(root=coco_root_train, ann_file=("./annotations/person_keypoints_train2017.json"), transform=transform)
person_keypoint_coco_dataset_val = create_coco_dataset(root=coco_root_val, ann_file=("./annotations/person_keypoints_val2017.json"), transform=transform)



# dataloader for each dataset
dataloader_train = DataLoader(coco_dataset_train, batch_size=32, shuffle=True, num_workers=4)
dataloader_val = DataLoader(coco_dataset_val, batch_size=32, shuffle=True, num_workers=4)
#dataloader_test = DataLoader(coco_root_test, batch_size=32, shuffle=True, num_workers=4)
#dataloader_unlabeled = DataLoader(coco_root_unlabeled, batch_size=32, shuffle=True, num_workers=4)

for batch in dataloader_train: 
    print(len(batch))
    images, targets = batch #traget means batch of labels
    
#for batch in dataloader_val:
    #images, targets = batch

#for batch in dataloader_test:
   # images, targets = batch
    
#for batch in dataloader_unlabeled:
    #images, targets = batch
    


loading annotations into memory...
Done (t=136.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!
loading annotations into memory...
Done (t=0.76s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=5.83s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


## show images and targets

## CNN feature extraction

In [7]:


# Define a simple CNN model for feature extraction
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Adjust input size based on your image dimensions
        self.fc2 = nn.Linear(256, 64)
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 128 * 16 * 16)  # Adjust input size based on your image dimensions
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the transform to be applied to the images
transform = transforms.Compose([
    transforms.ToTensor(),
    # Add more transformations as needed
])

# Initialize the model
model = SimpleCNN()

# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model for a few epochs (this is a simplified training loop)
num_epochs = 5
for epoch in range(num_epochs):
    for images, targets in dataloader_train:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

# Feature extraction example (using the first batch of the dataloader)
with torch.no_grad():
    example_images, _ = next(iter(dataloader_train))
    extracted_features = model(example_images)
    print("Shape of extracted features:", extracted_features.shape)

NameError: name 'dataloader_train' is not defined

 ## DETR

In [ ]:
class MyDataset(nn.Module):
    def __init__(self, root, ann_file, transform=None):
        self.root = root
        self.transform = transform
        self.ann_file = ann_file
        self.coco = CocoDetection(root=root, annFile=ann_file, transform=transform)
        self.coco_info = self.coco.coco

    def __getitem__(self, idx):
        img, target = self.coco[idx]
        img_id = self.coco_info.imgs[idx]
        ann_ids = self.coco_info.getAnnIds(imgIds=img_id)
        anns = self.coco_info.loadAnns(ann_ids)
        return img, target, anns

    def __len__(self):
        return len(self.coco)